# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

In [1]:
import pandas as pd
file1 = 'raw_data/schools_complete.csv' 
file2 = 'raw_data/students_complete.csv'

df_school = pd.read_csv(file1)
df_student = pd.read_csv(file2)

## District Summary

In [2]:
df_summary = pd.DataFrame(
    {
        'Total Schools':15,
        'Total Students': '{:,d}'.format(df_school['size'].sum()),
        'Total Budget': '${:,.2f}'.format(df_school['budget'].sum()),
        'Average Math Score': df_student['math_score'].mean(),
        'Average Reading Score': df_student['reading_score'].mean(),
        '% Passing Math': df_student.loc[df_student['math_score']>=71,:]['math_score'].count()/df_student['name'].count()*100,
        '% Passing Reading': df_student.loc[df_student['reading_score']>=71,:]['reading_score'].count()/df_student['name'].count()*100,
        '% Overall Passing Rate': (df_student.loc[df_student['math_score']>=71,:]['math_score'].count()/df_student['name'].count()+df_student.loc[df_student['reading_score']>=71,:]['reading_score'].count()/df_student['name'].count())/2*100
    }, index=[0]
)
df_summary

,% Overall Passing Rate,% Passing Math,% Passing Reading,Average Math Score,Average Reading Score,Total Budget,Total Schools,Total Students
0,77.681899,72.392137,82.971662,78.985371,81.87784,"$24,649,428.00",15,"39,170"


## School Summary

In [3]:
df_school_sum = df_school.rename(columns={'name':'school','type':'School Type','size':'Total Students','budget':'Total Budget'})
df_school_sum['Per Student Budget'] = df_school_sum['Total Budget'] / df_school_sum['Total Students']

In [4]:
df_avgmath = pd.DataFrame(df_student.groupby(['school'])['math_score'].mean()).reset_index()
df_school_sum = pd.merge(df_avgmath, df_school_sum, on="school", how="right")

In [5]:
df_school_sum = df_school_sum.rename(columns={'math_score':'Average Math Score'})

In [6]:
df_avgread = pd.DataFrame(df_student.groupby(['school'])['reading_score'].mean()).reset_index()
df_school_sum = pd.merge(df_avgread, df_school_sum, on="school", how="right")

In [7]:
df_school_sum = df_school_sum.rename(columns={'reading_score':'Average Reading Score'})

In [8]:
df_math_70 = df_student[df_student['math_score']>70]
df_math_pass = df_math_70.groupby(['school'])['math_score'].count() / df_student.groupby(['school'])['math_score'].count()
df_math_pass = pd.DataFrame(df_math_pass).reset_index()

In [9]:
df_school_sum = pd.merge(df_math_pass, df_school_sum, on="school", how="right")

In [10]:
df_school_sum = df_school_sum.rename(columns={'math_score':'% Passing Math'})

In [11]:
df_read_70 = df_student[df_student['reading_score']>70]
df_read_pass = df_read_70.groupby(['school'])['reading_score'].count() / df_student.groupby(['school'])['reading_score'].count()
df_read_pass = pd.DataFrame(df_read_pass).reset_index()

In [12]:
df_school_sum = pd.merge(df_read_pass, df_school_sum, on="school", how="right")
df_school_sum = df_school_sum.rename(columns={'reading_score':'% Passing Reading'})

In [13]:
df_school_sum['% Overall Passing Rate'] = (df_school_sum['% Passing Reading'] + df_school_sum['% Passing Math'])/2

In [14]:
df_school_sum['% Passing Reading'] = df_school_sum['% Passing Reading'] * 100
df_school_sum['% Passing Math'] = df_school_sum['% Passing Math'] * 100
df_school_sum['% Overall Passing Rate'] = df_school_sum['% Overall Passing Rate'] * 100

In [15]:
df_school_sum['Total Budget'] = df_school_sum['Total Budget'].map("${:.2f}".format)
df_school_sum['Per Student Budget'] = df_school_sum['Per Student Budget'].map("${:.2f}".format)

In [16]:
df_school_sum

,school,% Passing Reading,% Passing Math,Average Reading Score,Average Math Score,School ID,School Type,Total Students,Total Budget,Per Student Budget,% Overall Passing Rate
0,Bailey High School,79.300643,64.630225,81.033963,77.048432,7,District,4976,$3124928.00,$628.00,71.965434
1,Cabrera High School,93.864370,89.558665,83.975780,83.061895,6,Charter,1858,$1081356.00,$582.00,91.711518
2,Figueroa High School,78.433367,63.750424,81.158020,76.711767,1,District,2949,$1884411.00,$639.00,71.091896
3,Ford High School,77.510040,65.753925,80.746258,77.102592,13,District,2739,$1763916.00,$644.00,71.631982
4,Griffin High School,93.392371,89.713896,83.816757,83.351499,4,Charter,1468,$917500.00,$625.00,91.553134
5,Hernandez High School,78.187702,64.746494,80.934412,77.289752,3,District,4635,$3022020.00,$652.00,71.467098
6,Holden High School,92.740047,90.632319,83.814988,83.803279,8,Charter,427,$248087.00,$581.00,91.686183
7,Huang High School,78.813850,63.318478,81.182722,76.629414,0,District,2917,$1910635.00,$655.00,71.066164
8,Johnson High School,78.281874,63.852132,80.966394,77.072464,12,District,4761,$3094650.00,$650.00,71.067003
9,Pena High School,92.203742,91.683992,84.044699,83.839917,9,Charter,962,$585858.00,$609.00,91.943867


## Top Performing Schools (By Passing Rate)

In [17]:
df_school_sum.sort_values(['% Overall Passing Rate'], ascending=False).head()

,school,% Passing Reading,% Passing Math,Average Reading Score,Average Math Score,School ID,School Type,Total Students,Total Budget,Per Student Budget,% Overall Passing Rate
13,Wilson High School,93.254490,90.932983,83.989488,83.274201,5,Charter,2283,$1319574.00,$578.00,92.093736
9,Pena High School,92.203742,91.683992,84.044699,83.839917,9,Charter,962,$585858.00,$609.00,91.943867
14,Wright High School,93.444444,90.277778,83.955000,83.682222,10,Charter,1800,$1049400.00,$583.00,91.861111
1,Cabrera High School,93.864370,89.558665,83.975780,83.061895,6,Charter,1858,$1081356.00,$582.00,91.711518
6,Holden High School,92.740047,90.632319,83.814988,83.803279,8,Charter,427,$248087.00,$581.00,91.686183


## Bottom Performing Schools (By Passing Rate)

In [18]:
df_school_sum.sort_values(['% Overall Passing Rate'], ascending=False).tail()

,school,% Passing Reading,% Passing Math,Average Reading Score,Average Math Score,School ID,School Type,Total Students,Total Budget,Per Student Budget,% Overall Passing Rate
5,Hernandez High School,78.187702,64.746494,80.934412,77.289752,3,District,4635,$3022020.00,$652.00,71.467098
2,Figueroa High School,78.433367,63.750424,81.158020,76.711767,1,District,2949,$1884411.00,$639.00,71.091896
8,Johnson High School,78.281874,63.852132,80.966394,77.072464,12,District,4761,$3094650.00,$650.00,71.067003
7,Huang High School,78.813850,63.318478,81.182722,76.629414,0,District,2917,$1910635.00,$655.00,71.066164
10,Rodriguez High School,77.744436,64.066017,80.744686,76.842711,11,District,3999,$2547363.00,$637.00,70.905226


## Math Scores by Grade

In [19]:
df_mathgrade = df_student.groupby(['grade','school'])['math_score'].mean().unstack(level=0)
#df_mathgrade = df_mathgrade.reset_index()
df_mathgrade.columns.name = None
df_mathgrade = df_mathgrade.reset_index()
df_mathgrade

,school,10th,11th,12th,9th
0,Bailey High School,76.996772,77.515588,76.492218,77.083676
1,Cabrera High School,83.154506,82.765560,83.277487,83.094697
2,Figueroa High School,76.539974,76.884344,77.151369,76.403037
3,Ford High School,77.672316,76.918058,76.179963,77.361345
4,Griffin High School,84.229064,83.842105,83.356164,82.044010
5,Hernandez High School,77.337408,77.136029,77.186567,77.438495
6,Holden High School,83.429825,85.000000,82.855422,83.787402
7,Huang High School,75.908735,76.446602,77.225641,77.027251
8,Johnson High School,76.691117,77.491653,76.863248,77.187857
9,Pena High School,83.372000,84.328125,84.121547,83.625455


## Reading Score by Grade 

In [20]:
df_readgrade = df_student.groupby(['grade','school'])['reading_score'].mean().unstack(level=0)
df_readgrade.columns.name = None
df_readgrade = df_readgrade.reset_index()
df_readgrade

,school,10th,11th,12th,9th
0,Bailey High School,80.907183,80.945643,80.912451,81.303155
1,Cabrera High School,84.253219,83.788382,84.287958,83.676136
2,Figueroa High School,81.408912,80.640339,81.384863,81.198598
3,Ford High School,81.262712,80.403642,80.662338,80.632653
4,Griffin High School,83.706897,84.288089,84.013699,83.369193
5,Hernandez High School,80.660147,81.396140,80.857143,80.866860
6,Holden High School,83.324561,83.815534,84.698795,83.677165
7,Huang High School,81.512386,81.417476,80.305983,81.290284
8,Johnson High School,80.773431,80.616027,81.227564,81.260714
9,Pena High School,83.612000,84.335938,84.591160,83.807273


## Scores by School Spending

In [21]:
bins = [0,585,615,645,675]
labels = ['<$585','$585-615','$615-645','$645-675']

In [22]:
df_spending = df_school_sum[['school','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate','Per Student Budget']]

In [23]:
df_spending['Per Student Budget'] = df_spending["Per Student Budget"].str.replace('$','')
df_spending['Per Student Budget'] = pd.to_numeric(df_spending['Per Student Budget'])
df_spending

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,school,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Per Student Budget
0,Bailey High School,77.048432,81.033963,64.630225,79.300643,71.965434,628.0
1,Cabrera High School,83.061895,83.975780,89.558665,93.864370,91.711518,582.0
2,Figueroa High School,76.711767,81.158020,63.750424,78.433367,71.091896,639.0
3,Ford High School,77.102592,80.746258,65.753925,77.510040,71.631982,644.0
4,Griffin High School,83.351499,83.816757,89.713896,93.392371,91.553134,625.0
5,Hernandez High School,77.289752,80.934412,64.746494,78.187702,71.467098,652.0
6,Holden High School,83.803279,83.814988,90.632319,92.740047,91.686183,581.0
7,Huang High School,76.629414,81.182722,63.318478,78.813850,71.066164,655.0
8,Johnson High School,77.072464,80.966394,63.852132,78.281874,71.067003,650.0
9,Pena High School,83.839917,84.044699,91.683992,92.203742,91.943867,609.0


In [24]:
df_spending["Range"] = pd.cut(df_spending["Per Student Budget"], bins, labels=labels)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
df_spending = df_spending.groupby(['Range']).mean()

In [26]:
del df_spending['Per Student Budget']

In [27]:
df_spending

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
Range,,,,,
<$585,83.455399,83.933814,90.350436,93.325838,91.838137
$585-615,83.599686,83.885211,90.788049,92.410786,91.599418
$615-645,79.079225,81.891436,73.021426,83.214343,78.117884
$645-675,76.997210,81.027843,63.972368,78.427809,71.200088


## Scores by School Size

In [28]:
bins = [0,1000,2000,5000]
labels = ['Small (<1000)','Medium (1000-2000)','Large (2000-5000)']

In [29]:
df_size = df_school_sum[['school','Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate','Total Students']]

In [30]:
df_size["Range"] = pd.cut(df_size['Total Students'], bins, labels=labels)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [31]:
df_size = df_size.groupby(['Range']).mean()
df_size

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate,Total Students
Range,,,,,,
Small (<1000),83.821598,83.929843,91.158155,92.471895,91.815025,694.500
Medium (1000-2000),83.374684,83.864438,89.931303,93.244843,91.588073,1704.400
Large (2000-5000),77.746417,81.344493,67.631335,80.190800,73.911067,3657.375


## Scores by School Type

In [32]:
df_type = df_school_sum.groupby('School Type').mean()

In [33]:
df_type = df_type[['Average Math Score','Average Reading Score','% Passing Math','% Passing Reading','% Overall Passing Rate']]

In [34]:
df_type

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing Rate
School Type,,,,,
Charter,83.473852,83.896421,90.363226,93.052812,91.708019
District,76.956733,80.966636,64.302528,78.324559,71.313543
